In [1]:
from nustar_pysolar import planning, io
import astropy.units as u

The 'recommended_units' attribute is deprecated since 3.0 and may be removed
in a future version. Its main use, of representing angles in degrees in frames,
is now done automatically in frames. Further overrides are discouraged but can
be done using a frame's ``frame_specific_representation_info``.
 [astropy.coordinates.representation]


# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='./data/')
print(fname)

./data/NUSTAR.2019_008.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='./data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

./data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [6]:
tstart = '2019-01-12T12:00:00'
tend = '2019-01-12T23:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [7]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 356.66450088180875 deg


/Users/bwgref/anaconda/lib/python3.5/site-packages/nustar_pysolar-0.1.dev164-py3.5.egg/nustar_pysolar/planning.py:189: SunpyDeprecationWarning: Use sunpy.coordinates.get_sun_P() for higher accuracy
  sun_np=sun.solar_north(t=time).deg * u.deg


# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [8]:
offset = [-190., -47.]*u.arcsec

# Loop over each orbit and correct the pointing for the same heliocentric pointing position.

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

In [9]:
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

/Users/bwgref/anaconda/lib/python3.5/site-packages/nustar_pysolar-0.1.dev164-py3.5.egg/nustar_pysolar/planning.py:133: SunpyDeprecationWarning: Use sunpy.coordinates.get_sun_P() for higher accuracy
  sun_np = sunpy.sun.solar_north(t=time).cgs


Orbit: 0
Orbit start: 2019-01-12T11:43:30 Orbit end: 2019-01-12T12:45:40
Aim time: 2019-01-12T12:14:35 RA (deg): 293.5430246884607 deg Dec (deg): -21.693292074963686 deg

Orbit: 1
Orbit start: 2019-01-12T13:20:10 Orbit end: 2019-01-12T14:22:10
Aim time: 2019-01-12T13:51:10 RA (deg): 293.61565606128727 deg Dec (deg): -21.682412919624156 deg

Orbit: 2
Orbit start: 2019-01-12T14:56:50 Orbit end: 2019-01-12T15:58:50
Aim time: 2019-01-12T15:27:50 RA (deg): 293.688322567184 deg Dec (deg): -21.671491251373528 deg

Orbit: 3
Orbit start: 2019-01-12T16:33:30 Orbit end: 2019-01-12T17:35:30
Aim time: 2019-01-12T17:04:30 RA (deg): 293.76097726656735 deg Dec (deg): -21.66053806738169 deg

Orbit: 4
Orbit start: 2019-01-12T18:10:10 Orbit end: 2019-01-12T19:12:10
Aim time: 2019-01-12T18:41:10 RA (deg): 293.83362012934873 deg Dec (deg): -21.64955339207573 deg

Orbit: 5
Orbit start: 2019-01-12T19:46:50 Orbit end: 2019-01-12T20:48:50
Aim time: 2019-01-12T20:17:50 RA (deg): 293.90625112708585 deg Dec (deg)